# 广义线性模型 Generalized Linear Models

引入科学计算和绘图相关包：

In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline

在线性回归的例子中，我们假设 $y|x;\theta \sim N(\mu, \sigma^2)$；而对于逻辑回归，我们假设 $y|x;\theta \sim b(1, \phi)$，也即伯努利分布。本节我们将看到，这两个模型都是广义线性模型的一种特殊形式。

本节包括以下内容：

1. 指数分布族 The exponential family
2. 构造广义线性模型 Constructing GLMs
3. 最小二乘法 Ordinary Least Squares
4. 逻辑回归 Logistic Regression
5. Softmax回归 Softmax Regression

### 1. 指数分布族 The exponential family

指数分布族定义如下：
$$ P(y;\eta) = b(y)exp(\eta^TT(y)-a(\eta)) $$

其中，$\eta$ 称为**自然参数 natural parameter**，$T(y)$ 称为**充分统计量 sufficient statistic**（在我们接下来将考虑的分布中，$T(y)=y$），$a(\eta)$ 称为**对数配分函数 log partition function**。$e^{-a(\eta)}$ 通常作为使分布 $p(y;\eta)$ 之和(对于离散型)或积分（对于连续型）为 $1$ 的正则化常数。

固定函数 $T, a, b$，可以得到以 $\eta$ 为参数的一族分布。随着 $\eta$ 的改变，我们可以得到同族的不同分布。

伯努利分布和正态分布都是指数分布族中的一族分布。

以伯努利分布为例：
$$
\begin{split}
p(y; \phi) &= \phi^y(1-\phi)^{1-y} \\
&= exp(ylog\phi + (1-y)log(1-\phi)) \\
&= exp((log(\frac{\phi}{1-\phi}))y + log(1-\phi)))
\end{split}
$$

这里，我们令 $\eta=log(\frac{\phi}{1-\phi})$，可以解出得到$\phi=\frac{1}{1+e^{-\eta}}$，正是sigmoid函数。

此外，令 $T(y) = y, a(\eta) = -log(1-\phi) = log(1+e^\eta), b(y) = 1$，即可得到上式。注意这里 $\eta$ 是常数，因而 $\eta^T = \eta$。

考虑正态分布，在我们推到线性回归的过程中，方差 $\sigma^2$ 的值对最终的 $\theta$ 和 $h_\theta(x)$并没有影响。为简化计算，我们设 $\sigma^2 = 1$：
$$
\begin{split}
p(y; \mu) &= \frac{1}{\sqrt{2\pi}}exp(-\frac{1}{2}(y-\mu)^2) \\
&= \frac{1}{\sqrt{2\pi}}exp(-\frac{1}{2}y^2) \cdot exp(\mu y-\frac{1}{2}\mu^2)
\end{split}
$$

令 $\eta=\mu, T(y)=y, a(\eta)=\frac{\mu^2}{2}=\frac{\eta^2}{2}, b(y)=\frac{1}{\sqrt{2\pi}}exp(-\frac{y^2}{2})$，可得上式。

此外，诸如多项式分布、泊松分布、伽马分布、指数分布、Beta分布、狄利克雷分布等等，都属于指数分布族。

### 2. 构造广义线性模型 Constructing GLMs

给定一组特征 $x$，我们希望预测随机变量 $y$。在这种情况下，构造一个广义线性模型，大致需要以下几个假设：

1. $y|x;\theta \sim ExponentialFamily(\eta)$。也即是说，给定 $x$ 和 $\theta$，假设$y$ 服从以 $\eta$ 为参数的某一种指数分布族。

2. 给定 $x$，我们的目标是预测 $T(y)$ 的期望值。在大多数例子中，$T(y)=y$。所以这意味着我们希望假设函数 $h$ 的输出符合条件 $h(x) = E[y|x]$。注意到，线性回归和逻辑回归都满足这个条件。

3. 自然参数 $\eta$ 和输入 $x$ 是线性相关的，即 $\eta = \theta^Tx$。（或者，如果 $\eta$ 是向量的话，$\eta_i=\theta_i^Tx$）

其中，第三点某种程度上可以视作我们的一个算法设计选择，而并非本质意义上的假设。通过这三个假设，构造出的广义线性模型，通常易于学习，对于预测不同分布的 $y$ 也常常十分有效。